In [1]:

import numpy as np
from qulacs import QuantumCircuit
from itertools import combinations
from mcr.equiv_check import (
    equivalence_check_via_mqt_qcec,
    equiv,
    pauli_bit_equivalence_check,
)
from mcr.gate_apply import (
    PauliBit,
    grouping,
    loop_optimization,
    set_clifford_to_qulacs,
    zhang_optimization,
    multiply_all,
)
import pickle

from mcr.mcr_optimize import find_nontrivial_swap, find_mcr
from opt_using_mcr import test_algorithm, mcr_swap, three_layer_nontrivial_swap

In [2]:
nqubits = 3
with open(f"unopt_{nqubits}.pickle", "rb") as f:
    seq = pickle.load(f)
data = []
for elem in seq:
    sgn = str(elem[1])[0]
    pauli_str = str(elem[1])[1:]
    if sgn == "+":
        data.append(PauliBit(pauli_str, np.pi / 4))
    else:
        assert sgn == "-", f"Unexpected sign: {sgn}"
        data.append(PauliBit(pauli_str, -np.pi / 4))
data.append(PauliBit("Z" * nqubits, -np.pi / 4))  # Add identity gate

In [3]:
seq_a = [PauliBit(ele.get_pauli_str(), ele.get_angle()) for ele in data]
clifford_lst, optimized_data = test_algorithm(seq_a)

🎉 Successful optimization using MCR! 24 -> 14
🎉 Successful optimization using MCR! 14 -> 0


In [5]:
equiv([[],seq_a],[clifford_lst, optimized_data])

True

In [23]:
optimized_data

[PauliBit(-0.7853981633974483*XXY),
 PauliBit(0.7853981633974483*XZX),
 PauliBit(0.7853981633974483*YYX),
 PauliBit(-0.7853981633974483*IIZ),
 PauliBit(0.7853981633974483*ZXZ),
 PauliBit(-0.7853981633974483*XZY),
 PauliBit(-0.7853981633974483*YYY),
 PauliBit(0.7853981633974483*XZX),
 PauliBit(0.7853981633974483*YYX),
 PauliBit(-0.7853981633974483*ZZZ)]

In [6]:
from mcr.gate_apply import set_clifford_to_qulacs
from qulacs.circuit import QuantumCircuitOptimizer as QCO
from qulacs import QuantumCircuit
circ = QuantumCircuit(nqubits)
circ = set_clifford_to_qulacs(circ, clifford_lst)
# for elem in data:
    # circ.merge_circuit(elem.convert_into_qulacs())
QCO().optimize(circ,nqubits)
circ.get_gate(0).get_matrix()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])